In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
import random
from bs4 import BeautifulSoup
import sqlite3
import inspect
import re
import time
from datetime import datetime

driver = webdriver.Chrome()
driver.get("https://x.com/")

for_unfollow = []

In [3]:
def days_to_now(specific_date_str):
    specific_date = datetime.strptime(specific_date_str, "%Y-%m-%dT%H:%M:%S.%fZ")
    current_date = datetime.now()
    difference = current_date - specific_date
    days_difference = difference.days
    return days_difference

def record_count():
    with sqlite3.connect('following.db') as connection:
        cursor = connection.cursor()
        try:
            cursor.execute("SELECT COUNT(*) FROM following_table")
            count = cursor.fetchone()[0]
        finally:
            cursor.close()
    return count

def get_record_at_position(n):
    with sqlite3.connect('following.db') as connection:
        cursor = connection.cursor()
        try:
            query = "SELECT user_name FROM following_table LIMIT 1 OFFSET ?"
            cursor.execute(query, (n - 1,))
            record = cursor.fetchone()
            if record:
                {}
            else:
                return False
        finally:
            cursor.close()
    return record[0]

def delete_record_at_position(n):
    with sqlite3.connect('following.db') as connection:
        cursor = connection.cursor()
        try:
            fetch_query = "SELECT rowid FROM following_table LIMIT 1 OFFSET ?"
            cursor.execute(fetch_query, (n - 1,))
            record = cursor.fetchone()
            if record:
                delete_query = "DELETE FROM following_table WHERE rowid = ?"
                cursor.execute(delete_query, (record[0],))
                connection.commit()
            else:
                return False
        finally:
            cursor.close()
    return True

def check_local_limitation_rate():
    try:
        element = driver.find_element(By.XPATH, "//*[text()='local_rate_limited']")
        return True
    except:
        pass

    try:
        button = driver.find_element(By.XPATH, "//button[.//span[text()='Retry']]")
        return True
    except:
        pass

    try:
        span_element = driver.find_element(By.XPATH, "//span[text()='Something went wrong. Try reloading.']")
        return True
    except:
        pass

    return False

def is_inactive(user_link, treshold = 150):
    driver.get(user_link + '/with_replies')
    time.sleep(5)
    while(check_local_limitation_rate()):
        print("We reached a LOCAL LIMITATION RATE AND NEED TO STOP FOR A WHILE")
        time.sleep(2 * 60)
        driver.get(user_link + '/with_replies')
        time.sleep(10)       

    elements = driver.find_elements(By.TAG_NAME, 'time')
    tweet_times = set()
    for element in elements:
        try:
            datetime_value = element.get_attribute('datetime')
            tweet_times.add(days_to_now(datetime_value))
        except:
            pass
    latest_tweet = min(tweet_times)
    if (latest_tweet > treshold):
        return True
    else:
        return False
    
def do_unfollow():
    try:
        wait = WebDriverWait(driver, random.randint(10, 20))
        unfollow_button = wait.until(
            EC.presence_of_element_located((By.XPATH, "//button[contains(., 'Following')]"))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", unfollow_button)
        time.sleep(5)  # Optional: Add a small delay to ensure the element is in view
        # Click the button using JavaScript
        driver.execute_script("arguments[0].click();", unfollow_button)
        time.sleep(3)
        confirm_unfollow = driver.find_element(By.CSS_SELECTOR, "button.css-175oi2r[data-testid='confirmationSheetConfirm'][role='button']")
        confirm_unfollow.click()
        time.sleep(random.randint(20, 60))
        return True
    except:
        return False
    
def sleep_time(i):
    if i % 10 == 0:
        return (random.randint(5, 30))  # 15 minutes
    elif i % 5 == 0:
        return (random.randint(5, 20))  # 5 minutes
    else:
        return (random.randint(5, 15))  # 1 minute
    
def main():
    num_deleted_items = 0
    itration = 0
    print('The number of users at the first of run is: ', record_count())
    while record_count() > 1:
        itration += 1
        user_link = get_record_at_position(1)
        user_name = user_link.split('/')[-1]
        try:
            if(is_inactive(user_link)):
                do_unfollow()
                num_deleted_items += 1
                print(num_deleted_items, ' - ', user_name,  '  following was deleted! *************')
            else:
                print(itration, ' - ', user_name, '  is active.')
        except:
            print('Page did not load correctly!!!')
            
        delete_record_at_position(1)
        time.sleep(sleep_time(itration))
    print('num_deleted_items  =  ', num_deleted_items)
 

if __name__ == '__main__':
    main()

The number of users at the first of run is:  721
We reached a LOCAL LIMITATION RATE AND NEED TO STOP FOR A WHILE
